In [1]:
import os
import dotenv
import sys
dotenv.load_dotenv()
sys.path.append(os.getenv('LIBRARY_PATH'))

import pandas as pd
import numpy as np

# Custom imports
from finance import implied_volatility, black_scholes

## Initial Checks

In [2]:
data = pd.read_csv(os.getenv('DATA_PATH'))
# full data check
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308472 entries, 0 to 1308471
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Timestamp        1308472 non-null  object 
 1   Stock Price      1255126 non-null  float64
 2   Symbol           1308472 non-null  object 
 3   Option Type      1308472 non-null  object 
 4   Strike Price     1308472 non-null  float64
 5   Ask Price        1308472 non-null  float64
 6   Bid Price        1308472 non-null  float64
 7   Expiration Date  1308472 non-null  object 
dtypes: float64(4), object(4)
memory usage: 79.9+ MB


## Removing all non-trading days

In [11]:
# Filter to exchange trading days (NYSE) - removes weekends & market holidays
import pandas_market_calendars as mcal

# ensure Timestamp and Expiration Date are datetime
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data['Expiration Date'] = pd.to_datetime(data['Expiration Date'])

start = data['Timestamp'].dt.date.min()
end = data['Timestamp'].dt.date.max()
print(f"Data covers {start} through {end}")

nyse = mcal.get_calendar('NYSE')
schedule = nyse.schedule(start_date=start, end_date=end)
trading_days = pd.to_datetime(schedule.index).date

# Filter rows whose date is in the NYSE trading days
data_trading = data[data['Timestamp'].dt.date.isin(trading_days)].copy()

print('Rows before:', len(data))
print('Rows after (NYSE trading days):', len(data_trading))
print('Rows removed:', len(data) - len(data_trading))
print("NaNs in 'Stock Price' after filtering:", data_trading['Stock Price'].isna().sum())

# quick missing summary
missing_summary_trading = pd.DataFrame({
    'Missing Values': data_trading.isnull().sum(),
    'Percentage Missing': (data_trading.isnull().sum() / len(data_trading)) * 100
}).sort_values('Missing Values', ascending=False)

print('\nMissing summary after filtering to trading days:')
print(missing_summary_trading)

Data covers 2024-08-09 through 2025-10-30
Rows before: 1237120
Rows after (NYSE trading days): 1161031
Rows removed: 76089
NaNs in 'Stock Price' after filtering: 0

Missing summary after filtering to trading days:
                 Missing Values  Percentage Missing
Timestamp                     0                 0.0
Stock Price                   0                 0.0
Symbol                        0                 0.0
Option Type                   0                 0.0
Strike Price                  0                 0.0
Ask Price                     0                 0.0
Bid Price                     0                 0.0
Expiration Date               0                 0.0


## Validating data quality

In [ ]:
# Check for Stock Price values encoded as empty arrays or empty strings
import inspect

def is_empty_array(x):
    return isinstance(x, (list, tuple, np.ndarray)) and len(x) == 0

def is_empty_string(x):
    return isinstance(x, str) and x.strip() == ''

mask_empty_array = data_trading['Stock Price'].apply(is_empty_array)
mask_empty_string = data_trading['Stock Price'].apply(is_empty_string)

count_empty_array = mask_empty_array.sum()
count_empty_string = mask_empty_string.sum()

print(f"Empty-array encoded Stock Price count: {count_empty_array}")
print(f"Empty-string encoded Stock Price count: {count_empty_string}")

# Show example rows (first 10) for each case
if count_empty_array:
    print('\nExamples of empty-array encoded rows:')
    display(data_trading.loc[mask_empty_array].head(10))
else:
    print('\nNo empty-array encoded rows found.')

if count_empty_string:
    print('Examples of empty-string encoded rows:')
    display(data_trading.loc[mask_empty_string].head(10))
else:
    print('No empty-string encoded rows found.')

# Combined mask
mask_combined = mask_empty_array | mask_empty_string
print('\nTotal problematic rows:', mask_combined.sum())
data_trading.info()

Empty-array encoded Stock Price count: 0
Empty-string encoded Stock Price count: 0

No empty-array encoded rows found.
No empty-string encoded rows found.

Total problematic rows: 0
<class 'pandas.core.frame.DataFrame'>
Index: 1161031 entries, 0 to 1237119
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   Timestamp        1161031 non-null  datetime64[ns]
 1   Stock Price      1161031 non-null  float64       
 2   Symbol           1161031 non-null  object        
 3   Option Type      1161031 non-null  object        
 4   Strike Price     1161031 non-null  float64       
 5   Ask Price        1161031 non-null  float64       
 6   Bid Price        1161031 non-null  float64       
 7   Expiration Date  1161031 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(4), object(2)
memory usage: 79.7+ MB


## Exporting cleaned MSTR data as parquet file

In [13]:
# Apply filtered trading-day dataframe as the main `data` and persist to disk
# This overwrites `data` in the notebook session with `data_trading` created earlier.
import pyarrow

data = data_trading
print('Assigned data = data_trading')

# quick sanity
data.info()

# attempt to save as parquet (fast and compact). Fall back to CSV if parquet not available.
out_parquet = 'MSTR_Options_cleaned.parquet'
out_csv = 'MSTR_Options_cleaned.csv'
try:
    data.to_parquet(out_parquet, engine='pyarrow')
    print(f'Saved filtered dataset to {out_parquet}')
except Exception as e:
    data.to_csv(out_csv, index=False)
    print(f'Parquet save failed ({e}); saved CSV to {out_csv}')


Assigned data = data_trading
<class 'pandas.core.frame.DataFrame'>
Index: 1161031 entries, 0 to 1237119
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   Timestamp        1161031 non-null  datetime64[ns]
 1   Stock Price      1161031 non-null  float64       
 2   Symbol           1161031 non-null  object        
 3   Option Type      1161031 non-null  object        
 4   Strike Price     1161031 non-null  float64       
 5   Ask Price        1161031 non-null  float64       
 6   Bid Price        1161031 non-null  float64       
 7   Expiration Date  1161031 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(4), object(2)
memory usage: 79.7+ MB
Saved filtered dataset to MSTR_Options_cleaned.parquet


In [29]:
data.query("`Strike Price` == 80 and `Expiration Date` == '2027-12-17'")

C:\Users\Saber\AppData\Local\Temp\ipykernel_3568\1184799159.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  data.query("`Strike Price` == 80 and `Expiration Date` == '2027-12-17'")


,Timestamp,Stock Price,Symbol,Option Type,Strike Price,Ask Price,Bid Price,Expiration Date
320910,2025-04-04,291.130,MSTR271217C00080000,call,80.0,232.20,232.00,2027-12-17
323383,2025-04-07,268.690,MSTR271217C00080000,call,80.0,228.00,212.05,2027-12-17
325878,2025-04-08,237.930,MSTR271217C00080000,call,80.0,202.00,182.00,2027-12-17
328381,2025-04-09,295.685,MSTR271217C00080000,call,80.0,253.90,234.00,2027-12-17
330907,2025-04-10,272.860,MSTR271217C00080000,call,80.0,234.00,216.00,2027-12-17
...,...,...,...,...,...,...,...,...
1229566,2025-10-24,289.345,MSTR271217P00080000,put,80.0,10.95,10.10,2027-12-17
1231438,2025-10-27,295.255,MSTR271217P00080000,put,80.0,11.90,9.85,2027-12-17
1233297,2025-10-28,282.615,MSTR271217P00080000,put,80.0,10.85,10.10,2027-12-17
1235160,2025-10-29,274.390,MSTR271217P00080000,put,80.0,11.80,10.40,2027-12-17


Contract Symbol identifies individual contracts by specific strike price and expiry date

In [41]:
grouped = data.groupby('Symbol')
grouped.head()

,Timestamp,Stock Price,Symbol,Option Type,Strike Price,Ask Price,Bid Price,Expiration Date
0,2024-08-09,135.370,MSTR240816C00080000,call,80.0,56.35,54.80,2024-08-16
1,2024-08-09,135.370,MSTR240816C00082000,call,82.0,54.45,52.75,2024-08-16
2,2024-08-09,135.370,MSTR240816C00100000,call,100.0,36.30,35.00,2024-08-16
3,2024-08-09,135.370,MSTR240816C00105000,call,105.0,32.15,30.25,2024-08-16
4,2024-08-09,135.370,MSTR240816C00108000,call,108.0,28.85,27.05,2024-08-16
...,...,...,...,...,...,...,...,...
1236359,2025-10-30,253.935,MSTR260417P00145000,put,145.0,9.50,8.85,2026-04-17
1236361,2025-10-30,253.935,MSTR260417P00155000,put,155.0,11.15,10.45,2026-04-17
1236413,2025-10-30,253.935,MSTR260515P00145000,put,145.0,11.30,10.50,2026-05-15
1236415,2025-10-30,253.935,MSTR260515P00155000,put,155.0,13.00,12.20,2026-05-15
